## Create Multiple Hardware Schemas

When a measurement is associated with multiple hardware entities, it may be desirable for each type of hardware to have its own metadata schema.  This demonstrates how to create multiple hardware entities with different schemas.

In [5]:
from ni.datastore.client import Client

client = Client()
# Register schemas
with open("cable_schema.toml", "r", encoding="utf-8") as f:
    cable_schema = f.read()
cable_schema_id = client.register_schema(cable_schema)
with open("socket_schema.toml", "r", encoding="utf-8") as f:
    socket_schema = f.read()
socket_schema_id = client.register_schema(socket_schema)
with open("scope_schema.toml", "r", encoding="utf-8") as f:
    scope_schema = f.read()
scope_schema_id = client.register_schema(scope_schema)

## Create the Hardware Objects with their Custom Metadata

Now that the schemas are registered, we can create Hardware objects that specify custom metadata to meet the requirements of the registered schemas.  Each piece of hardware can use it's own schema.

In [6]:
from ni.measurements.metadata.v1.metadata_store_pb2 import HardwareItem, ExtensionValue

cable = HardwareItem(
    manufacturer="NI",
    model="cable",
    serial_number="7u2349",
    schema_id=cable_schema_id,
)
cable.extensions["cable_length"].CopyFrom(ExtensionValue(string_value="1.5"))
cable.extensions["manufacture_date"].CopyFrom(ExtensionValue(string_value="2023-01-01"))

socket = HardwareItem(
    manufacturer="NI",
    model="socket",
    schema_id=socket_schema_id,
)
socket.extensions["socket_number"].CopyFrom(ExtensionValue(string_value="3"))
socket.extensions["manufacture_date"].CopyFrom(ExtensionValue(string_value="2024-05-01"))

scope = HardwareItem(
    manufacturer="NI",
    model="PXIe-5171",
    serial_number="1933B4E",
    schema_id=scope_schema_id,
)
scope.extensions["bandwidth"].CopyFrom(ExtensionValue(string_value="250 MHz"))
scope.extensions["manufacture_date"].CopyFrom(ExtensionValue(string_value="2024-11-03"))

## Create a Schema for the Test Result (optional)

If you specify a schema for the test result, it will ensure that all the hardware custom metadata is a superset of whatever metadata is specified in the session schema.  This allows the author to ensure all hardware has some consistent attributes.


In [7]:
with open("test_result_schema.toml", "r", encoding="utf-8") as f:
    test_result_schema = f.read()
test_result_schema_id = client.register_schema(test_result_schema)

## Publish a Measurement with the Hardware

Now that all the schemas have been registered and the hardware objects have been created, we can publish our data with the associated hardware.

In [8]:
from datetime import datetime, timedelta, timezone
from ni.measurements.data.v1.data_store_pb2 import TestResult, Step
from ni.protobuf.types.precision_timestamp_pb2 import PrecisionTimestamp
from nitypes.waveform import AnalogWaveform
from nitypes.waveform import Timing
import numpy as np

cable_id = client.create_hardware_item(hardware_item=cable)
socket_id = client.create_hardware_item(hardware_item=socket)
scope_id = client.create_hardware_item(hardware_item=scope)

test_result_id = client.create_test_result(
    test_result=TestResult(
        test_result_name="scope measurements",
        hardware_item_ids=[cable_id, socket_id, scope_id],
        schema_id=test_result_schema_id
    )
)

precision_timestamp = PrecisionTimestamp()
waveform = AnalogWaveform(
    sample_count=3,
    raw_data=np.array([1.0, 2.0, 3.0]),
    timing=Timing.create_with_regular_interval(
        timedelta(seconds=1e-3),
        datetime.now(timezone.utc)
    )
)

step = Step(step_name="Initial step", test_result_id=test_result_id)
step_id = client.create_step(step)
published_measurement = client.publish_measurement(
    measurement_name="scope reading",
    value=waveform,
    step_id=step_id,
)

# Query for the Published Data

Now that we've published some data, we can use our OData query API to find it

In [9]:
published_measurements = client.query_measurements("$filter=name eq 'scope reading'")
found_measurement = next(iter(published_measurements), None)
if found_measurement is not None:
    waveform = client.read_data(found_measurement.moniker, expected_type=AnalogWaveform)
    print(f"published data is: {waveform.raw_data}")

published data is: [1. 2. 3.]
